In [ ]:
# Python
import pandas as pd
import numpy as np

#
# Python
#

#
import importlib
import utilities.variables as variables
import utilities.utility as utility
import utilities.plots as plots
import utilities.prophet_utils as prophet_utils
import utilities.mpt_utils as mpt_utils
importlib.reload(variables)
importlib.reload(utility)
importlib.reload(plots)
importlib.reload(prophet_utils)
#
import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

In [ ]:
# Includes by default percentage values
df = pd.read_csv('../../../data/df_monthly_returns_complete.csv', index_col='Date')
df_pct = pd.read_csv('../../../data/df_monthly_returns_complete_percentage.csv', index_col='Date')

### Train & Plot Methods

In [ ]:
#filtered_cols = [col for col in df.columns if not '.T' in col]
#df = df[filtered_cols][600:800]
df_pct.drop(columns=['2181.T', 'BGEO.L', 'ILM1.DE', 'MCG.L', '8766.T'], inplace=True)

In [ ]:
test_size_years = variables.FIVE_YEARS_NR
# actual values
df_train = df_pct.head(int(variables.ALL_YEARS_NR - test_size_years) * 12)
df_test = df_pct.tail(test_size_years * 12)

### 1 Month Actual vs Prediction

In [ ]:
importlib.reload(prophet_utils)
importlib.reload(plots)

forecasts_1m = prophet_utils.train_predict(dataframe=df_train, months=1)
#
plots.plot_lines_actual_vs_predicted(dataframe=df_pct, forecasts=forecasts_1m, months=1)

#### Overview table - 1 Month

In [ ]:
importlib.reload(prophet_utils)
importlib.reload(mpt_utils)

df_pred_1m, raw_weights_1m, mu_1m, S_1m, sigma_1m, sharpe_1m = mpt_utils.get_prediction_portfolio_performance(forecasts_1m, "prophet_weights_1m.csv", min_avg_return=0, months=12)
mpt_utils.create_discrete_allocation(df, raw_weights_1m, total_portfolio_value = 10000)

# Create overview
df_view_1m = pd.DataFrame.from_dict(raw_weights_1m, orient='index', columns=['max_sharpe_weight'])
# Extract volatilities (square root of diagonal elements)
df_view_1m['avg_annual_volatility'] = pd.Series(np.sqrt(np.diag(S_1m)), index=S_1m.columns).values
# Set annual returns
df_view_1m['avg_annual_return'] = mu_1m.values
#
df_view_1m['return_last_period(1m)'] = round(df_test.head(1).prod() - 1, 2)
df_view_1m.head(10)

### 6 Months Actual vs Prediction

In [ ]:
importlib.reload(prophet_utils)

forecasts_6m = prophet_utils.train_predict(dataframe=df_train, months=6)
#
prophet_utils.plot(dataframe=df_pct, forecasts=forecasts_6m, months=6)


In [ ]:
importlib.reload(prophet_utils)
importlib.reload(mpt_utils)

df_pred_6m, raw_weights_6m, mu_6m, S_6m, sigma_6m, sharpe_6m = mpt_utils.get_prediction_portfolio_performance(forecasts_6m, "prophet_weights_6m.csv", min_avg_return=0, months=12)
mpt_utils.create_discrete_allocation(df, raw_weights_6m)

In [ ]:
importlib.reload(prophet_utils)

df_view_6m = pd.DataFrame.from_dict(raw_weights_6m, orient='index', columns=['max_sharpe_weight'])
# Extract volatilities (square root of diagonal elements)
df_view_6m['avg_annual_volatility'] = pd.Series(np.sqrt(np.diag(S_6m)), index=S_6m.columns).values
# Set annual returns
df_view_6m['avg_annual_return'] = mu_6m.values
#
df_view_6m['return_last_period(6m)'] = round(df_test.head(6).prod() - 1, 2)
df_view_6m

### 12 months Actual vs Prediction

In [ ]:
importlib.reload(prophet_utils)
importlib.reload(plots)

# TODO change to 12 instead of 11
forecasts_12m = prophet_utils.train_predict(dataframe=df_train, months=12)
#
plots.plot_lines_actual_vs_predicted(dataframe=df_pct, forecasts=forecasts_12m, months=12)

In [ ]:
importlib.reload(prophet_utils)
importlib.reload(mpt_utils)

# @TODO Check issue with 12 months, maybe it gets out of range, 11 Months works fine
#
df_pred_12m, raw_weights_12m, mu_12m, S_12m, sigma_12m, sharpe_12m = mpt_utils.get_prediction_portfolio_performance(forecasts_12m, "prophet_weights_12m.csv", min_avg_return=0, months=12)
mpt_utils.create_discrete_allocation(df, raw_weights_12m)

df_view_12m = pd.DataFrame.from_dict(raw_weights_12m, orient='index', columns=['max_sharpe_weight'])
# Extract volatilities (square root of diagonal elements)
df_view_12m['avg_annual_volatility'] = pd.Series(np.sqrt(np.diag(S_12m)), index=S_12m.columns).values
# Set annual returns
df_view_12m['avg_annual_return'] = mu_12m.values
#
df_view_12m['return_last_period(12m)'] = round(df_test.head(12).prod() - 1, 2)
df_view_12m.head(10)